In [3]:
#%pip install wget
#%pip install torch
import data_rnn
import numpy as np
import torch

In [4]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = data_rnn.load_imdb(final=False)

Inspecting the data

In [5]:
x_train[0], y_train[0] # 0 is positive, 1 is negative

([14, 19, 9, 379, 22, 11, 50, 52, 53, 290], 1)

In [17]:
[i2w[word] for word in x_train[0]]
len(i2w)

99430

In [13]:
lengths = [len(l) for l in x_train]
print(np.min(lengths), np.max(lengths), np.average(lengths))

10 2514 240.6318


In [20]:
w2i['.pad'], w2i['.start'], w2i['.end'], w2i['.unk']

(0, 1, 2, 3)

In [24]:
def pad(seq, pad_length):
    padded = np.zeros(pad_length) # 0 is for padding
    padded[0:len(seq)] = seq
    return torch.tensor(padded, dtype=torch.long)
    

In [51]:
padded = pad(x_train[0], 12).reshape(1,-1)
padded.shape

torch.Size([1, 12])

In [52]:
# Take a tensor x, and return max across time dimension
def MaxPoolTime(x):
    return torch.amax(x, dim=1)

class q2_baseline(torch.nn.Module):
    def __init__(self):
        super().__init__()
        batch_size = 1
        timestep = 12
        numcls = 2
        hidden = 300
        embedding_size = 300
        n_embeddings = len(i2w)
        self.emb = torch.nn.Embedding(n_embeddings, embedding_size, padding_idx=0)
        self.fc1 = torch.nn.Linear(embedding_size, hidden)
        self.fc2 = torch.nn.Linear(hidden, numcls)
    
    def forward(self, x):
        x = self.emb(x)

        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        
        print(f"x before:{x.size()}")
        x = MaxPoolTime(x)
        print(f"x after:{x.size()}")
        x = self.fc2(x)

        return x

In [53]:
rnn = q2_baseline()
y = rnn.forward(padded)
torch.nn.functional.softmax(y)

x before:torch.Size([1, 12, 300])
x after:torch.Size([1, 300])


/var/folders/zw/n1g26qmn72ldwxh52bkgncv40000gn/T/ipykernel_9233/4214664088.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(y)


tensor([[0.4269, 0.5731]], grad_fn=<SoftmaxBackward0>)